# SIC Quick Start Tutorial (v0.1 2022-11-10)

Welcome to the SIC Quick Start Tutorial!
Below is a brief introduction to the SIC Python package and its main features.
Just follow the instructions and run the code cells to get started.

## 1. Installation

The SIC package is available on PyPI and can be installed via pip:

```bash
pip install sic42
```

Evaluate the following cell to check it by clicking on the cell and pressing `Shift+Enter` or the `Run` button in the toolbar above.

In [ ]:
# Install the SIC package from the test PyPI repository
%%capture
%pip install -i https://test.pypi.org/pypi/ --extra-index-url https://pypi.org/simple sic42==0.2.0

We need some additional files for this tutorial, so let's get them first from the GitHub Repo. Let's also remove the sample_data folder and move the content of the files from the GitHub Repo to the working directory. (Not needed in case you use the package locally.)

In [ ]:
# Remove the sample data folder and clone the template repository from GitHub 
# Move the files from the template repository to the current directory
%%capture
!rm -rf /content/sample_data
!git clone https://github.com/lab42-global/sic42-template.git
!mv /content/sic42-template/* /content/
!rm -rf /content/sic42-template


## 2. Importing and testing the package

Let's import the package and test it using the behavior scripts that are included in the package.

In [ ]:
# Test the installation by running the tournament using the template agents in the package
# A folder named "deathmatch_results" should be created in the current directory
# In there you will find the results, the logs and the replays of the tournament
from sic42 import sic
sic.Tournament().run_tournament()

Let's check if the visualization works by running the following cell:

In [ ]:
from IPython.display import HTML
from base64 import b64encode
video_path = '/content/deathmatch_results/round_1/game_1_Eater_Reproducer/set_1/visualization.mp4'

mp4 = open(video_path,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src={decoded_vid} type="video/mp4"></video>')

## 3. Using your own behavior scripts and configuration files

To use your own scripts and configuration files, you need to create a new folder to put the scripts in. We called it "competitors" and already copied it above from the GitHub Repo. Note that there should only be behavior scripts in this folder, no other files. The configuration files are in the main directory (or another folder you can speciy). Addditionaly, make sure that there are enough behavior scripts in the folder for the current game mode. E.g., for the deathmatch mode, there should be at least 2 behavior scripts in the folder (as the agents defined by them play against each other).

Let's test this using the copied files from the GitHub Repo. (The behavior scripts are the same as those that come with the package.)

In [ ]:
# If you want to run the tournament with your own agents, you can do so by providing the paths to the config and the competitors files
from sic42 import sic
config_path = '/content/deathmatch_config.json'
competitors_path = '/content/competitors'
tournament = sic.Tournament(
        config_path=config_path,
        competitors_path=competitors_path
    )
tournament.run_tournament()

If the simulation was successful you should now find a folder called "deathmatch_results" containing the results of the simulation. The folder contains the final leaderboard, the log files of the the tournament and one folder per round that has been played. 

Each round folder is split into folders for each game, named after the competitors. Each folder contains a csv with the results of each game played between the two competitors as well as the time they used. Additionally there is one folder per set that has been played between the competitors. In there you can find the visualizations of the games as well as the log files and plots (if specified they should be created in the config file).

In summary here is the folder structure of the results folder (if you use the default config files and the default oponents from the GitHub Repo (called "Eater" and "Reproducer")):

```
deathmatch_results
├── leaderboard.csv
├── tournament_log.json
├── round_1
│   ├── game_1_Reproducer_Eater
│   │   ├── results.csv
│   │   ├── times.csv
│   │   ├── set_1
│   │   │   ├── visualization.mp4
│   │   │   ├── logs
│   │   │   │   ├── Eater_actions.json
│   │   │   │   ├── Eater_errors.json
│   │   │   │   ├── Reproducer_actions.json
│   │   │   │   ├── Reproducer_errors.json
│   │   │   ├── plots
│   │   │   │   ├── action_counts
│   │   │   │   │   ├── attack_count.png
│   │   │   │   │   ├── eat_count.png
│   │   │   │   │   ├── ...
│   │   │   │   ├── average_agent_attribute_values
│   │   │   │   │   ├── average_energy_level.png
│   │   │   │   │   ├── ...
│   │   ├── set_2
│   │   │   ├── ...
│   ├── game_2_X_Y
│   │   ├── ...
│   ├── ...
├── round_2
│   ├── ...
```


## 4. Creating your own behavior scripts

Creating your own behavior script is simple. Just create a new python file in the folder you created above and define a main function that is called by the simulation. The input the agent receives will be stored in a JSON file called "agent_input.json" in the same folder as the behavior script. The output of the agent should be stored in a JSON file called "agent_output.json" in the same folder as the behavior script. 

An easy way to read the "agent_input.json" is to use the same function as we have in our sample scripts called read_from_json:
    
```python
import json

def from_json(path='agent_input.json'):
    with open(path, 'r') as fp:
        json_obj = json.load(fp)
    return (
        json_obj['self_view'],
        np.array(json_obj['relative_indices']),
        np.array(json_obj['entities']),
        np.array(json_obj['pheromones'], dtype='object')
    )
```

The output of the agent should be a list of action name, action value pairs. The action name should be a string and the action value is e.g. a list of numbers. As an example an output that moves the agent to the right would look like this:

```json
[
    ["step", [1, 0]]
]
```

### 4.1 A simple example

Let's create a simple behavior script that just moves the agent to the right and does nothing else. We will call it "forward.py" and put it in the "competitors" folder. The script should look like this:

```python
import json
import sys
sys.path.append('...')

def from_json(path='agent_input.json'):
    with open(path, 'r') as fp:
        json_obj = json.load(fp)
    return (
        json_obj['self_view'],
        np.array(json_obj['relative_indices']),
        np.array(json_obj['entities']),
        np.array(json_obj['pheromones'], dtype='object')
    )

def main():
    """ simply move right """
    self_view, indices, entities, pheromones = from_json() # read input from json even though we don't need it
    # Initialize output list containing the desired actions
    desired_actions = []
    # Move Right
    desired_actions.append(('step', [0, 1]))
    # Write output to json
    with open('agent_output.json', 'w') as fp:
        json.dump(desired_actions, fp)
```

Now let's make a file out of that and copy it to the competitors folder before running the simulation again. Let's do that in two steps:

1. Create the file and copy it to the competitors folder (actually we create two files here, one for a left and one for a right moving agent to fill up the tournament roster)
2. Run the simulation again

You can do that by running the following two cells:

In [ ]:
# Define the above Python code as a string
python_code_right = """
import json
import sys
import numpy as np
sys.path.append('...')

def from_json(path='agent_input.json'):
    with open(path, 'r') as fp:
        json_obj = json.load(fp)
    return (
        json_obj['self_view'],
        np.array(json_obj['relative_indices']),
        np.array(json_obj['entities']),
        np.array(json_obj['pheromones'], dtype='object')
    )

def main():
    \"\"\" simply move right \"\"\"
    self_view, indices, entities, pheromones = from_json() # read input from json even though we don't need it
    # Initialize output list containing the desired actions
    desired_actions = []
    # Move Right
    desired_actions.append(('step', [0, 1]))
    # Write output to json
    with open('agent_output.json', 'w') as fp:
        json.dump(desired_actions, fp)

# Call main function
if __name__ == '__main__':
    main()
"""

# Also create a left mover agent by changing the corresponding line in the code
python_code_left = python_code_right.replace('Move Right', 'Move Left')
python_code_left = python_code_left.replace('[0, 1]', '[0, -1]')

import os
# Get the current working directory
cwd = os.getcwd()
# Specify the filename
right_filename = "RightMover.py"
left_filename = "LeftMover.py"

# Check if the competitors folder exists, if not, create it
competitors_folder = 'competitors'
competitors_folder = os.path.join(cwd, competitors_folder)
if not os.path.exists(competitors_folder):
    os.makedirs(competitors_folder)

# Specify the full path
right_file_path = os.path.join(competitors_folder, right_filename)
left_filen_path = os.path.join(competitors_folder, left_filename)

# Write the right mover agent to a Python file
with open(right_file_path, 'w') as file:
    file.write(python_code_right)
    
# Write the left mover agent to a Python file
with open(left_filen_path.replace('Right', 'Left'), 'w') as file:
    file.write(python_code_left)

print(f"Python files written to {competitors_folder}")


In [ ]:
# Run the tournament including the new agent
from sic42 import sic
config_path = '/content/deathmatch_config.json'
competitors_path = '/content/competitors'
tournament = sic.Tournament(
        config_path=config_path,
        competitors_path=competitors_path
    )
tournament.run_tournament()

You should now find the corresponding visualisations and log files in the results folder. Note that the left and right moving agents die quickly as they are eating anything to replenish their energy.

In [ ]:
from IPython.display import HTML
from base64 import b64encode
video_path = '/content/deathmatch_results/round_1/game_1_RightMover_Reproducer/set_1/visualization.mp4'

mp4 = open(video_path,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src={decoded_vid} type="video/mp4"></video>')

## 5. Changing or Creating your own configuration files

For a start you can ignore most of the options in the configuration file. But maybe you would like to increase the number of time steps of each game?

You can do that by opening the deathmatch_config.json which you find on the same level as this notebook (you should see a folder icon on the left side of the notebook). Then you can change the value of the "n_timesteps" option under "simulation" to e.g. 128. Then you can run the simulation again and see that the games now take longer.

Similarly you can change the other options in the config file. See the documentation for more information on the options.

Let's test that and change the number of time steps to 64. You can do that by running the following two cells:

In [ ]:
import json

# Define the path to the configuration file
config_file_path = '/content/deathmatch_config.json'

# Load the current configuration from the file
with open(config_file_path, 'r') as file:
    config_data = json.load(file)

# Update the 'n_timesteps' value under 'simulation' to 74
config_data['simulation']['n_timesteps'] = 64

# Write the updated configuration back to the file
with open(config_file_path, 'w') as file:
    json.dump(config_data, file, indent=4)

# Confirm the change was made (optional)
print("Updated 'n_timesteps' to:", config_data['simulation']['n_timesteps'])

In [ ]:
# Run the tournament including the new agent
from sic42 import sic
config_path = '/content/deathmatch_config.json'
competitors_path = '/content/competitors'
tournament = sic.Tournament(
        config_path=config_path,
        competitors_path=competitors_path
    )
tournament.run_tournament()

If everything worked as intended the simulation should now run for 64 time steps instead of 32. We can check the video again which should now run for 8 seconds instead of 4.

In [ ]:
from IPython.display import HTML
from base64 import b64encode
video_path = '/content/deathmatch_results/round_1/game_1_RightMover_Reproducer/set_1/visualization.mp4'

mp4 = open(video_path,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src={decoded_vid} type="video/mp4"></video>')